In [1]:
# Import libraries
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
import seaborn as sns
import random
import scipy
from scipy.stats import chi2

In [10]:
# Read the data
df = pd.read_csv('data/MC3_survival.csv', sep='\t')

# # Filter by cancer type
# df=df[df['type']=='LUAD']

# Formatting the gender
df['gender'] = df['gender'].replace(to_replace='FEMALE', value=0) # FEMALE -> 0
df['gender'] = df['gender'].replace(to_replace='MALE', value=1) # MALE -> 1

df['ajcc_pathologic_tumor_stage'] = df['ajcc_pathologic_tumor_stage'].replace(to_replace='Stage I', value=1)
df['ajcc_pathologic_tumor_stage'] = df['ajcc_pathologic_tumor_stage'].replace(to_replace='Stage IA', value=1)
df['ajcc_pathologic_tumor_stage'] = df['ajcc_pathologic_tumor_stage'].replace(to_replace='Stage IB', value=1)
df['ajcc_pathologic_tumor_stage'] = df['ajcc_pathologic_tumor_stage'].replace(to_replace='Stage II', value=2)
df['ajcc_pathologic_tumor_stage'] = df['ajcc_pathologic_tumor_stage'].replace(to_replace='Stage IIA', value=2)
df['ajcc_pathologic_tumor_stage'] = df['ajcc_pathologic_tumor_stage'].replace(to_replace='Stage IIB', value=2)
df['ajcc_pathologic_tumor_stage'] = df['ajcc_pathologic_tumor_stage'].replace(to_replace='Stage IIC', value=2)

df['ajcc_pathologic_tumor_stage'] = df['ajcc_pathologic_tumor_stage'].replace(to_replace='Stage III', value=3)
df['ajcc_pathologic_tumor_stage'] = df['ajcc_pathologic_tumor_stage'].replace(to_replace='Stage IIIA', value=3)
df['ajcc_pathologic_tumor_stage'] = df['ajcc_pathologic_tumor_stage'].replace(to_replace='Stage IIIB', value=3)
df['ajcc_pathologic_tumor_stage'] = df['ajcc_pathologic_tumor_stage'].replace(to_replace='Stage IIIC', value=3)
df['ajcc_pathologic_tumor_stage'] = df['ajcc_pathologic_tumor_stage'].replace(to_replace='Stage IV', value=4)
df['ajcc_pathologic_tumor_stage'] = df['ajcc_pathologic_tumor_stage'].replace(to_replace='Stage IVA', value=4)
df['ajcc_pathologic_tumor_stage'] = df['ajcc_pathologic_tumor_stage'].replace(to_replace='Stage IVB', value=4)
df['ajcc_pathologic_tumor_stage'] = df['ajcc_pathologic_tumor_stage'].replace(to_replace='Stage IVC', value=4)
df['ajcc_pathologic_tumor_stage'] = df['ajcc_pathologic_tumor_stage'].replace(to_replace='[Not Available]', value=0)
df['ajcc_pathologic_tumor_stage'] = df['ajcc_pathologic_tumor_stage'].replace(to_replace='[Not Applicable]', value=0)
df['ajcc_pathologic_tumor_stage'] = df['ajcc_pathologic_tumor_stage'].replace(to_replace='[Discrepancy]', value=0)


df

/tmp/ipykernel_218804/2837699575.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['gender'] = df['gender'].replace(to_replace='MALE', value=1) # MALE -> 1


,bcr_patient_barcode,X,type,age_at_initial_pathologic_diagnosis,gender,race,ajcc_pathologic_tumor_stage,clinical_stage,histological_type,histological_grade,...,OS,OS.time,DSS,DSS.time,DFI,DFI.time,PFI,PFI.time,Redaction,leuk.frac
0,TCGA-02-0001,2646,GBM,44.0,0,WHITE,0,[Not Applicable],Untreated primary (de novo) GBM,[Not Available],...,1.0,358.0,1.0,358.0,NaN,NaN,1.0,137.0,NaN,0.228080
1,TCGA-02-0003,2647,GBM,50.0,1,WHITE,0,[Not Applicable],Untreated primary (de novo) GBM,[Not Available],...,1.0,144.0,1.0,144.0,NaN,NaN,1.0,40.0,NaN,0.047688
2,TCGA-02-0006,2649,GBM,56.0,0,WHITE,0,[Not Applicable],Untreated primary (de novo) GBM,[Not Available],...,1.0,558.0,1.0,558.0,NaN,NaN,1.0,302.0,NaN,0.255156
3,TCGA-02-0007,2650,GBM,40.0,0,WHITE,0,[Not Applicable],Treated primary GBM,[Not Available],...,1.0,705.0,1.0,705.0,NaN,NaN,1.0,518.0,NaN,0.047660
4,TCGA-02-0009,2651,GBM,61.0,0,WHITE,0,[Not Applicable],Untreated primary (de novo) GBM,[Not Available],...,1.0,322.0,1.0,322.0,NaN,NaN,1.0,264.0,NaN,0.064487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10683,TCGA-ZS-A9CF,5801,LIHC,64.0,1,WHITE,2,[Not Applicable],Hepatocellular Carcinoma,G2,...,0.0,2412.0,0.0,2412.0,1.0,636.0,1.0,636.0,NaN,0.059373
10684,TCGA-ZS-A9CF,5801,LIHC,64.0,1,WHITE,2,[Not Applicable],Hepatocellular Carcinoma,G2,...,0.0,2412.0,0.0,2412.0,1.0,636.0,1.0,636.0,NaN,0.057896
10685,TCGA-ZS-A9CG,5802,LIHC,55.0,1,WHITE,2,[Not Applicable],Hepatocellular Carcinoma,G2,...,0.0,341.0,0.0,341.0,0.0,341.0,0.0,341.0,NaN,0.133980
10686,TCGA-ZU-A8S4,1953,CHOL,52.0,1,WHITE,1,[Not Applicable],Cholangiocarcinoma; intrahepatic,G3,...,1.0,98.0,1.0,98.0,1.0,72.0,1.0,72.0,NaN,0.465039


In [11]:
# Reduced data frame with the desired parameters
cox_df=df[['age_at_initial_pathologic_diagnosis', 'ajcc_pathologic_tumor_stage' ,'OS.time','gender','OS', 'type']]

cox_df = pd.get_dummies(cox_df, columns=['type'], drop_first=True)


# Removing rows with NAs
cox_df = cox_df[cox_df['OS.time'].notna()]
cox_df = cox_df[cox_df['age_at_initial_pathologic_diagnosis'].notna()]

# Fitting the model and printing the results
cph = CoxPHFitter()
cph.fit(cox_df, duration_col = 'OS.time', event_col='OS')
cph.print_summary()

ValueError: could not convert string to float: '[Unknown]'

In [80]:
# Reduced data frame with the desired parameters
cox_df=df[['leuk.frac', 'gender', 'age_at_initial_pathologic_diagnosis','PFI.time','PFI']]

# Removing rows with NAs
cox_df = cox_df[cox_df['PFI.time'].notna()]
cox_df = cox_df[cox_df['age_at_initial_pathologic_diagnosis'].notna()]

# Fitting the model and printing the results
cph = CoxPHFitter()
cph.fit(cox_df, duration_col = 'PFI.time', event_col='PFI')
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 521 total observations, 304 right-censored observations>
             duration col = 'PFI.time'
                event col = 'PFI'
      baseline estimation = breslow
   number of observations = 521
number of events observed = 217
   partial log-likelihood = -1200.52
         time fit was run = 2023-03-15 14:53:23 UTC

---
                                      coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                                   
leuk.frac                            -0.10       0.91       0.53            -1.13             0.93                 0.32                 2.54
gender                                0.19       1.21       0.14            -0.08             0.46                 0.93                 1.58
age_at_initial_pathologic_diagnosis  -0.00       1.00       0.01            -0.02             0.01                 0.98                 1.01

                                      cmp to     z    p   -log2(p)
covariate                                                         
leuk.frac                               0.00 -0.18 0.85       0.23
gender                                  0.00  1.40 0.16       2.63
age_at_initial_pathologic_diagnosis     0.00 -0.37 0.71       0.49
---
Concordance = 0.55
Partial AIC = 2407.04
log-likelihood ratio test = 2.21 on 3 df
-log2(p) of ll-ratio test = 0.92